In [3]:
import pandas as pd

In [75]:
costs = (
    pd
    .read_csv('../data/tech_params.csv')
    .drop(['CO2 intensity'],axis=1)
    .set_index('technology')
    .fillna(0)
)

costs['overnight_capex'] = costs['investment']

#costs = costs.drop(['investment'],axis=1)

costs.head(5)

,efficiency,FOM,fuel,lifetime,VOM,construction_time,capital_spend_year_1,capital_spend_year_2,capital_spend_year_3,capital_spend_year_4,capital_spend_year_5,capital_spend_year_6,capital_spend_year_7,overnight_capex
technology,,,,,,,,,,,,,,
battery inverter,0.900,3.000000,0.0,20.0,0.0,1,1.00,0.00,0.0,0.0,0.0,0.0,0.0,411.0
battery storage,0.000,0.000000,0.0,15.0,0.0,1,1.00,0.00,0.0,0.0,0.0,0.0,0.0,192.0
biomass,0.468,4.526935,7.0,30.0,0.0,4,0.15,0.35,0.3,0.2,0.0,0.0,0.0,2209.0
gas - ccgt,0.500,2.500000,21.6,30.0,4.0,3,0.30,0.60,0.1,0.0,0.0,0.0,0.0,800.0
gas - ocgt,0.000,0.000000,0.0,0.0,0.0,2,0.75,0.25,0.0,0.0,0.0,0.0,0.0,0.0


## Calculating annualised capital costs
There are [three types of investment costs](https://atb.nrel.gov/electricity/2023/data) that we need to consider: overnight capital costs, capital costs and annualised costs. The technology investment costs in the dataframe above are **overnight** capital costs in $\frac{\text{USD}}{\text{MW}}$. Overnight cost is the cost of a construction project if no interest was incurred during construction, as if the project was completed "overnight." We therefore have to convert these to capital costs by factoring in the costs incurred during the construction phase of the project. To do so, we use the following formula: 

$$
c_x = k . c_o
$$

where $c_x$ is the capital cost (or CAPEX), $c_o$ is the overnight capital cost and $k$ is the construction finance factor [defined](https://atb.nrel.gov/electricity/2023/data) as:

$$
k = \sum_{y=0}^{Y} { h_y . [ 1 + ((1+IDC)^{y + 0.5} - 1) ] }
$$

where $h_y$ is the fraction of capital spent in year $y$ accounting for the interest during construction (IDC) over the whole construction phase. Once we have the capital costs, we can then calculate the annualised capital cost ($c_{a}$, USD/MW/annum) by multiplying the capital costs by the annuity factor $a$, such that 

$$
c_{a} = a . c_x 
$$

where

$$
a = \frac{1-(1+r)^{-n}}{r}
$$

where $r$ is the discount rate and $n$ is the lifetime of the specific asset. 

## A worked example: let's compute annualised costs

In [69]:
# convert overnight costs to capital costs
def calculate_construction_finance_factor(
        r : float,      # discount rate [0-1]
        y : list,       # years ( e.g., [1,2,3] )
        c : list,       # fractional capital distribution of technology (e.g., [0.5,0.5,0]). Must sum to equal 1.
):
    if isinstance(y, list) and isinstance(c, list):
        return \
            pd.Series( [ (1 + ((1+r)**(i+0.5)-1)) for i in y] ) * c
    else:
        return \
            (1 + ((1+r)**(y+0.5)-1) ) * c

In [80]:
# let's calculate the capex

y = list( range(1,len(costs.filter(regex='capital_spend').columns)+1) )

# calculate construction finance factor (k) for each technology
costs['k'] = ( 
    costs
    .apply( 
        lambda row: \
            calculate_construction_finance_factor( 
                r = 0.1, 
                y = y, 
                c = row.filter(regex='capital_spend').to_list(),
            ), axis=1
    ).sum(axis=1)
)

# calculate capex
(costs
 .assign(capex = costs['overnight_capex'] * costs['k'])
 #.drop( ['construction_time', 'k'], axis=1)
 .drop( costs.filter(regex='capital_spend').columns, axis=1 )
)

,efficiency,FOM,fuel,lifetime,VOM,construction_time,overnight_capex,k,capex
technology,,,,,,,,,
battery inverter,0.900,3.000000,0.0,20.0,0.0,1,411.0,1.153690,474.166480
battery storage,0.000,0.000000,0.0,15.0,0.0,1,192.0,1.153690,221.508429
biomass,0.468,4.526935,7.0,30.0,0.0,4,2209.0,1.343126,2966.964422
gas - ccgt,0.500,2.500000,21.6,30.0,4.0,3,800.0,1.247139,997.710881
gas - ocgt,0.000,0.000000,0.0,0.0,0.0,2,0.0,1.182532,0.000000
gas - chp,0.000,0.000000,0.0,0.0,0.0,3,0.0,1.247139,0.000000
coal,0.464,1.923076,8.4,40.0,6.0,5,1300.0,1.390900,1808.169843
electrolysis,0.800,4.000000,0.0,18.0,0.0,1,350.0,1.153690,403.791407
fuel cell,0.580,3.000000,0.0,20.0,0.0,1,339.0,1.153690,391.100819


In [82]:
# calculate capex and tidy up dataframe
costs_adj = ( 
    costs
    .assign(capex = costs['overnight_capex'] * costs['k'])
    .drop( ['construction_time', 'k'], axis=1)
    .drop( costs.filter(regex='capital_spend').columns, axis=1 )
)

costs_adj.head(5)

,efficiency,FOM,fuel,lifetime,VOM,overnight_capex,capex
technology,,,,,,,
battery inverter,0.900,3.000000,0.0,20.0,0.0,411.0,474.166480
battery storage,0.000,0.000000,0.0,15.0,0.0,192.0,221.508429
biomass,0.468,4.526935,7.0,30.0,0.0,2209.0,2966.964422
gas - ccgt,0.500,2.500000,21.6,30.0,4.0,800.0,997.710881
gas - ocgt,0.000,0.000000,0.0,0.0,0.0,0.0,0.000000


### Finally, let's annualise these costs as above

In [83]:
def calculate_annuity(
        n : int, 
        r : float,
) -> float:
    '''

    Calculate the annuity factor for an asset with lifetime n years and
    discount rate of r, e.g. annuity(20, 0.05) * 20 = 1.6
    
    Inputs:
    -----------------------------------

        n : asset lifetime (years)
        r : discount rate (%, decimal point [e.g., 0.04])

    '''

    if isinstance(r, pd.Series):
        return pd.Series(1/n, index=r.index).where(r == 0, r/(1. - 1./(1.+r)**n))
    elif r > 0:
        return r/(1. - 1./(1.+r)**n)
    else:
        return 1/n

In [88]:
costs_adj['annualised_capex'] = (
    calculate_annuity(costs_adj['lifetime'], r = 0.1) * costs_adj['capex']
)

costs_adj.dropna()

,efficiency,FOM,fuel,lifetime,VOM,overnight_capex,capex,annualised_capex
technology,,,,,,,,
battery inverter,0.900,3.000000,0.0,20.0,0.0,411.0,474.166480,55.695417
battery storage,0.000,0.000000,0.0,15.0,0.0,192.0,221.508429,29.122550
biomass,0.468,4.526935,7.0,30.0,0.0,2209.0,2966.964422,314.733355
gas - ccgt,0.500,2.500000,21.6,30.0,4.0,800.0,997.710881,105.836420
coal,0.464,1.923076,8.4,40.0,6.0,1300.0,1808.169843,184.902389
electrolysis,0.800,4.000000,0.0,18.0,0.0,350.0,403.791407,49.234376
fuel cell,0.580,3.000000,0.0,20.0,0.0,339.0,391.100819,45.938556
H2 pipeline,0.980,5.000000,0.0,40.0,0.0,267.0,308.035159,31.499495
HVAC overhead,0.000,2.000000,0.0,40.0,0.0,400.0,461.475893,47.190255


## Calculating marginal costs
The marginal cost $c_m$ is a function of the variable operational and maintenance costs ($c_{vom}$), fuel costs ($c_f$) and the efficiency $e$, such that:

$$
c_m = c_{vom} + \frac{c_{f}}{e}
$$

In [90]:
# now let's compute the marginal cost
costs_adj["marginal_cost"] = (costs_adj["VOM"] + costs_adj["fuel"] / costs_adj["efficiency"]).fillna(0)

#costs_adj["marginal_cost_uom"] = 'USD/MWh'

costs_adj.head(5)

,efficiency,FOM,fuel,lifetime,VOM,overnight_capex,capex,annualised_capex,marginal_cost,marginal_cost_uom
technology,,,,,,,,,,
battery inverter,0.900,3.000000,0.0,20.0,0.0,411.0,474.166480,55.695417,0.000000,USD/MWh
battery storage,0.000,0.000000,0.0,15.0,0.0,192.0,221.508429,29.122550,0.000000,USD/MWh
biomass,0.468,4.526935,7.0,30.0,0.0,2209.0,2966.964422,314.733355,14.957265,USD/MWh
gas - ccgt,0.500,2.500000,21.6,30.0,4.0,800.0,997.710881,105.836420,47.200000,USD/MWh
gas - ocgt,0.000,0.000000,0.0,0.0,0.0,0.0,0.000000,NaN,0.000000,USD/MWh


In [91]:
# calculating captial costs of storage is a little trickier... let's do that now
def costs_for_storage(
        # calculate cost of storages
        store, link1, link2=None, max_hours=1.0
):
    capital_cost = link1["annualised_capex"] + max_hours * store["annualised_capex"]
    if link2 is not None:
        capital_cost += link2["annualised_capex"]
    return pd.Series(
        dict(annualised_capex=capital_cost, marginal_cost=0.0)
    )

# compute storage costs
costs_adj.loc["battery"] = costs_for_storage(
    costs_adj.loc["battery storage"],
    costs_adj.loc["battery inverter"],
    max_hours=4, # assuming 4-hr battery storage
)

# compute H2 storage costs
costs_adj.loc["H2"] = costs_for_storage(
    costs_adj.loc["hydrogen underground storage"],
    costs_adj.loc["fuel cell"],
    costs_adj.loc["electrolysis"],
    max_hours=168, # assuming 168-hr H2 storage
)

costs_adj.loc[['battery','H2']]

,efficiency,FOM,fuel,lifetime,VOM,overnight_capex,capex,annualised_capex,marginal_cost,marginal_cost_uom
technology,,,,,,,,,,
battery,NaN,NaN,NaN,NaN,NaN,NaN,NaN,172.185616,0.0,NaN
H2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,105.082885,0.0,NaN


### Now, let's assume we are costing up a 10 GW solar farm and apply a social discount rate
We assume the marginal cost of solar is zero and therefore we ignore them.

In [102]:
plant_capacity = 10e3 # MW
n_years = 2030-2024 # years

# get solar capex (USD/MW/annum)
solar_capex = costs_adj.loc['solar'].annualised_capex

# compute annual cost (USD/annum)
total_annual_cost =  solar_capex * plant_capacity 

# convert to millions
total_annual_cost *= 1e-6

# calculate cumulative cost (assuming uniform distribution of capital)
cumulative_cost = total_annual_cost * n_years

# convert to present value applying a social discount rate
social_discount_rate = 0.1

npv = cumulative_cost / (1 + social_discount_rate)**(n_years)

print(f'Cumulative cost of {int(plant_capacity)} MW plant: {round(npv,1)} billion')

Cumulative cost of 10000 MW plant: 1.8 billion
